**COMPUTER VISION**\
**ASSIGNMENT-3**

**Referring Image Segmentation (RIS)**

Tanmay Singh\
2021569\
CSAI\
Class of '25

**Importing the Dependencies**

In [1]:
%pip install h5py
%pip install tqdm
%pip install timm
%pip install ftfy
%pip install scipy
%pip install regex
%pip install torch
%pip install gdown
%pip install requests
%pip install filelock
%pip install tokenizers
%pip install safetensors
%pip install torchvision
%pip install transformers
%pip install scikit-image
%pip install huggingface_hub
%pip install mmcv-full==1.3.12
%pip install pycocotools==2.0.2
%pip install mmsegmentation==0.17.0
%pip install opencv-python==4.5.3.56
!git clone https://github.com/yz93/LAVT-RIS.git

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you 

In [2]:
%cd LAVT-RIS

/teamspace/studios/this_studio/CV_A3/LAVT-RIS


In [3]:
%pip install -r requirements.txt

  Using cached pycocotools-2.0.2.tar.gz (23 kB)
  Preparing metadata (setup.py) ... done
  Using cached opencv-python-4.5.3.56.tar.gz (89.2 MB)
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [975 lines of output]
      Ignoring numpy: markers 'python_version == "3.6" and platform_machine != "aarch64" and platform_machine != "arm64"' don't match your environment
      Ignoring numpy: markers 'python_version >= "3.6" and sys_platform == "linux" and platform_machine == "aarch64"' don't match your environment
      Ignoring numpy: markers 'python_version >= "3.6" and sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_machine != "aarch64" and platform_machine != "arm64"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.8" and

In [1]:
import os
import gdown
import torch
import zipfile
import warnings
import numpy as np
warnings.filterwarnings("ignore")

from PIL import Image
import torch.nn.functional as F

import torchvision.transforms as T
from bert.tokenization_bert import BertTokenizer

from bert.modeling_bert import BertModel
from lib import segmentation
import matplotlib.pyplot as plt


/home/nsl/cv/a3/LAVT-RIS/venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Downloading the Pre-trained weights**

In [2]:
# Change drive link to ReCoCo weights and add to ./checkpoints dir
drive_link = "https://drive.google.com/uc?id=1m-PDAQs0n_XLPpqm3xHBrbBqO3TsbEhh"

gdown.download(drive_link, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1m-PDAQs0n_XLPpqm3xHBrbBqO3TsbEhh
From (redirected): https://drive.google.com/uc?id=1m-PDAQs0n_XLPpqm3xHBrbBqO3TsbEhh&confirm=t&uuid=e18ec6d8-2654-48b9-9e20-e652ce7d2af5
To: /home/nsl/cv/a3/LAVT_RIS.pth
100%|██████████| 451M/451M [00:18<00:00, 24.4MB/s] 


'LAVT_RIS.pth'

**Downloading the Samples and the Reference Text**

In [3]:
download_dir = "data.zip"
drive_link = "https://drive.google.com/uc?id=1DvOTDrzEhDx44CLf_qcoS7XsVG8MBP5f"

gdown.download(drive_link, output = download_dir, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1DvOTDrzEhDx44CLf_qcoS7XsVG8MBP5f
To: /home/nsl/cv/a3/data.zip
100%|██████████| 248k/248k [00:00<00:00, 924kB/s]


'data.zip'

In [4]:
data_dir = "data"

os.makedirs(data_dir, exist_ok=True)

with zipfile.ZipFile(download_dir, 'r') as zipped:
    zipped.extractall(data_dir)

In [5]:
drive_link = "https://drive.google.com/uc?id=199i0BkKeJsaVube4zEoLPEfXZl0NagPQ"

gdown.download(drive_link, data_dir+"/reference.txt", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=199i0BkKeJsaVube4zEoLPEfXZl0NagPQ
To: /home/nsl/cv/a3/data/reference.txt
100%|██████████| 677/677 [00:00<00:00, 2.52MB/s]


'data/reference.txt'

In [6]:
weights = './checkpoints/refcoco.pth'
device = 'cuda:0'

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
def load_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img_ndarray = np.array(img)
    original_w, original_h = img.size

    image_transforms = T.Compose(
        [
        T.Resize(480),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]
    )
    img = image_transforms(img).unsqueeze(0)
    img = img.to(device)
    return img, img_ndarray, original_w, original_h

In [9]:
def tokenize_sentence(sentence):
    sentence_tokenized = tokenizer.encode(text=sentence, add_special_tokens=True)
    sentence_tokenized = sentence_tokenized[:20]
    
    padded_sent_toks = [0] * 20
    padded_sent_toks[:len(sentence_tokenized)] = sentence_tokenized
    
    attention_mask = [0] * 20
    attention_mask[:len(sentence_tokenized)] = [1]*len(sentence_tokenized)
    
    padded_sent_toks = torch.tensor(padded_sent_toks).unsqueeze(0)
    attention_mask = torch.tensor(attention_mask).unsqueeze(0)
    padded_sent_toks = padded_sent_toks.to(device)
    attention_mask = attention_mask.to(device)
    return padded_sent_toks, attention_mask

In [10]:
class args:
    swin_type = 'base'
    window12 = True
    mha = ''
    fusion_drop = 0.0

In [11]:
def load_model(weights):
    single_model = segmentation.__dict__['lavt'](pretrained='', args=args)
    single_model.to(device)

    model_class = BertModel
    single_bert_model = model_class.from_pretrained('bert-base-uncased')
    single_bert_model.pooler = None

    checkpoint = torch.load(weights, map_location='cpu')
    single_bert_model.load_state_dict(checkpoint['bert_model'])
    single_model.load_state_dict(checkpoint['model'])
    model = single_model.to(device)
    bert_model = single_bert_model.to(device)
    return model, bert_model

In [12]:
def plot_classification_feature_map(logits_tensor, original_size, save_path):
    probs = torch.softmax(logits_tensor, dim=1)  # (1, C, 480, 480)
    max_confidence, _ = probs.max(dim=1)  # (1, 480, 480)

    # Resize to original image size
    max_confidence_resized = F.interpolate(max_confidence.unsqueeze(1), size=original_size, mode='bilinear', align_corners=False)
    heatmap = max_confidence_resized.squeeze().detach().cpu().numpy()  # (H, W)

    plt.figure(figsize=(6, 6))
    plt.title("Filled Classification Confidence Map")
    plt.imshow(heatmap, cmap='plasma')
    plt.colorbar(label='Confidence')
    plt.axis('off')
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

In [13]:
def get_output(img, padded_sent_toks, attention_mask, original_h, original_w, model, bert_model, plot_feature_map=True, save_path=None):
    last_hidden_states = bert_model(padded_sent_toks, attention_mask=attention_mask)[0]
    embedding = last_hidden_states.permute(0, 2, 1)
    output = model(img, embedding, l_mask=attention_mask.unsqueeze(-1))

    if plot_feature_map:
        plot_classification_feature_map(output, original_size=(original_h, original_w), save_path=save_path)

    output = output.argmax(1, keepdim=True)
    output = F.interpolate(output.float(), (original_h, original_w))
    output = output.squeeze()
    output = output.cpu().data.numpy()
    return output

In [14]:
def overlay_davis(image, mask, colors=[[0, 0, 0], [255, 0, 0]], cscale=1, alpha=0.4):
    from scipy.ndimage.morphology import binary_dilation

    colors = np.reshape(colors, (-1, 3))
    colors = np.atleast_2d(colors) * cscale

    im_overlay = image.copy()
    object_ids = np.unique(mask)

    for object_id in object_ids[1:]:
        foreground = image*alpha + np.ones(image.shape)*(1-alpha) * np.array(colors[object_id])
        binary_mask = mask == object_id

        im_overlay[binary_mask] = foreground[binary_mask]

        countours = binary_dilation(binary_mask) ^ binary_mask
        im_overlay[countours, :] = 0

    return im_overlay.astype(image.dtype)

In [15]:
model, bert_model = load_model(weights)

Window size 12!
Randomly initialize Multi-modal Swin Transformer weights.


In [16]:
samples = os.listdir(os.path.join(data_dir, 'samples'))
n_samples = len(samples)
n_samples, samples

(10,
 ['ILSVRC2012_test_00000030.jpg',
  'ILSVRC2012_test_00000018.jpg',
  'ILSVRC2012_test_00000019.jpg',
  'ILSVRC2012_test_00000025.jpg',
  'ILSVRC2012_test_00000026.jpg',
  'ILSVRC2012_test_00000022.jpg',
  'ILSVRC2012_test_00000023.jpg',
  'ILSVRC2012_test_00000034.jpg',
  'ILSVRC2012_test_00000004.jpg',
  'ILSVRC2012_test_00000003.jpg'])

In [17]:
with open(os.path.join(data_dir, 'reference.txt'), 'r') as f:
    sentences = f.readlines()
sentences = [sentence.strip() for sentence in sentences]
sentences_map = {}
for s in sentences:
    img, caption = s.split(" : ")
    sentences_map[img] = caption.replace('\"', '')
sentences_map

{'ILSVRC2012_test_00000003.jpg': 'The walking dog in the picture',
 'ILSVRC2012_test_00000004.jpg': 'The smiling dog in the grass',
 'ILSVRC2012_test_00000018.jpg': 'The boy on left smiling and holding icecream',
 'ILSVRC2012_test_00000019.jpg': 'The black gray bird on in the picture',
 'ILSVRC2012_test_00000022.jpg': 'The sad dog standing beside the pool',
 'ILSVRC2012_test_00000023.jpg': 'The guy in white shirt on the bicycle',
 'ILSVRC2012_test_00000025.jpg': 'The butterfly in the picture',
 'ILSVRC2012_test_00000026.jpg': 'The mang wearing a suite and tie',
 'ILSVRC2012_test_00000030.jpg': 'The duck in the picture',
 'ILSVRC2012_test_00000034.jpg': 'The white coffee cups on the coffee machine'}

In [21]:
os.makedirs('output', exist_ok=True)
for i in range(n_samples):
    image_path = os.path.join(data_dir, 'samples', samples[i])
    sentence = sentences_map[samples[i]]

    print(f"Processing {samples[i]} with caption: {sentence}")

    img, img_ndarray, original_w, original_h = load_image(image_path)
    padded_sent_toks, attention_mask = tokenize_sentence(sentence)
    output = get_output(img, padded_sent_toks, attention_mask, original_h, original_w, model, bert_model, plot_feature_map=True, 
                        save_path=os.path.join('output', samples[i].replace('.jpg', '_feature_map.jpg')))
    output = output.astype(np.uint8)  # (orig_h, orig_w), np.uint8

    visualization = overlay_davis(img_ndarray, output)
    visualization = Image.fromarray(visualization)
    visualization.save(os.path.join('output', samples[i].replace('.jpg', '_result.jpg')))

Processing ILSVRC2012_test_00000030.jpg with caption: The duck in the picture
Processing ILSVRC2012_test_00000018.jpg with caption: The boy on left smiling and holding icecream
Processing ILSVRC2012_test_00000019.jpg with caption: The black gray bird on in the picture
Processing ILSVRC2012_test_00000025.jpg with caption: The butterfly in the picture
Processing ILSVRC2012_test_00000026.jpg with caption: The mang wearing a suite and tie
Processing ILSVRC2012_test_00000022.jpg with caption: The sad dog standing beside the pool
Processing ILSVRC2012_test_00000023.jpg with caption: The guy in white shirt on the bicycle
Processing ILSVRC2012_test_00000034.jpg with caption: The white coffee cups on the coffee machine
Processing ILSVRC2012_test_00000004.jpg with caption: The smiling dog in the grass
Processing ILSVRC2012_test_00000003.jpg with caption: The walking dog in the picture
